In [1]:
import tskit
import pandas
import numpy

from tqdm import tqdm
pandas.options.display.max_columns=999
import tsconvert
import functools
from functools import reduce
from scipy.stats import chi2_contingency

import warnings
warnings.filterwarnings('ignore')

Let's make an tree annotated by lineage (using shapes at the end of branches) and whether or not the sample contains an S450L mutation in rpoB (by dots around the circle)

First, let's read in the `GENOMES` table and extract the form of `uid` that Kerri used in her phylogenetic tree (this was how the vcf files were named but there were two problems so when I made the tables I fixed the problems, but because she used this original identifier we have to rebuilt it from the VCF filename)

In [3]:
GENOMES = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/GENOMES.pkl.gz')
GENOMES.reset_index(inplace=True)

def create_original_uid(row):

    filename = row.FTP_FILENAME_VCF

    if '.regeno' in filename:
        if filename[:5] != 'comas':
            return('site'+filename.split('/site')[1].split('.regeno')[0])
        else:
            return(filename.split('comas_regeno/')[1].split('.regeno')[0])
    else:
        return None

GENOMES['ORIGINAL_UID'] = GENOMES.apply(create_original_uid, axis=1)
GENOMES.set_index('UNIQUEID', inplace=True)
GENOMES


,SITEID,SUBJID,LABID,ISOLATENO,SEQREPS,BELONGS_GPI,PER_SAMPLE_VCF_PRESENT,REGENOTYPED_VCF_PRESENT,CLOCKWORK_VERSION,TBI_INDEX,KMER_COUNTS,SNP_DISTANCE_TO_H37rV,SPECIES,LINEAGE_NAME,SUBLINEAGE_NAME,LINEAGE_PERCENTAGE,N_NULL,N_SNP,N_INDEL,N_FILTER_FAIL,N_REF,N_HET,CATALOGUE_NAME,CATALOGUE_VERSION,TB_TYPE_1,WGS_PREDICTION_STRING,IMAGE_MD5SUM,FTP_PATH,FTP_FILENAME_VCF,TREE_PATH,TREE_FILENAME_VCF,FASTQ_MD5SUMS,SEQTREAT_SAMPLE,MYKROBE_LINEAGE_NAME_1,MYKROBE_LINEAGE_NAME_2,ENA,ORIGINAL_UID
UNIQUEID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
site.02.subj.0958.lab.22A197.iso.1,02,0958,22A197,1,197,True,True,True,0.8.3,True,False,1154.0,M. tuberculosis,Lineage 2,,71.283784,4934,0,1154,104,0,14329,CRyPTIC,v1.31,MDR,RRURRRSSSSSSSSS,{'02-0958-22A197-1-14': 'a587bac9ad2a0ebd36274...,/well/bag/jeffk/release_staging/,00/01/41/00/14100/site.02.iso.1.subject.0958.l...,dat/CRyPTIC2/V2/02/0958/22A197/1/regenotyped/,site.02.subj.0958.lab.22A197.iso.1.v0.8.3.rege...,,False,Lineage 2,lineage2.2.1,ERS5301054,site.02.iso.1.subject.0958.lab_id.22A197.seq_r...
site.02.subj.0823.lab.2013241494.iso.1,02,0823,2013241494,1,241494,True,True,True,0.8.3,True,False,388.0,M. tuberculosis,Lineage 4,,95.705521,2250,0,388,43,0,9442,CRyPTIC,v1.31,UNK,UUSUSSSSSSSSSSS,{'02-0823-2013241494-1-14': '698507bed7ff19268...,/well/bag/jeffk/release_staging/,00/01/41/43/14143/site.02.iso.1.subject.0823.l...,dat/CRyPTIC2/V2/02/0823/2013241494/1/regenotyped/,site.02.subj.0823.lab.2013241494.iso.1.v0.8.3....,,False,Lineage 2,lineage2.2.3,ERS5301097,site.02.iso.1.subject.0823.lab_id.2013241494.s...
site.02.subj.0359.lab.222018-14.iso.1,02,0359,222018-14,1,14222018,True,True,True,0.8.3,True,False,1147.0,M. tuberculosis,Lineage 2,,95.608108,3578,0,1147,118,0,13038,CRyPTIC,v1.31,UNK,SUSSSSSSUSSSSSS,{'02-0359-222018-14-1-14': '39c28529c7564ce379...,/well/bag/jeffk/release_staging/,00/01/08/73/10873/site.02.iso.1.subject.0359.l...,dat/CRyPTIC2/V2/02/0359/222018-14/1/regenotyped/,site.02.subj.0359.lab.222018-14.iso.1.v0.8.3.r...,,False,Lineage 2,lineage2.2.3,ERS5298526,site.02.iso.1.subject.0359.lab_id.222018-14.se...
site.02.subj.0224.lab.2013221088.iso.1,02,0224,2013221088,1,13221088_2013221088,True,True,True,0.8.3,True,False,1193.0,M. tuberculosis,Lineage 2,,96.621622,3529,0,1193,104,0,13201,CRyPTIC,v1.31,SUS,SSSSSSSSSSSSSSS,,/well/bag/jeffk/release_staging/,00/01/13/37/11337/site.02.iso.1.subject.0224.l...,dat/CRyPTIC2/V2/02/0224/2013221088/1/regenotyped/,site.02.subj.0224.lab.2013221088.iso.1.v0.8.3....,,False,Lineage 2,lineage2.2.5,ERS5300527,site.02.iso.1.subject.0224.lab_id.2013221088.s...
site.02.subj.0918.lab.22A153.iso.1,02,0918,22A153,1,153,True,True,True,0.8.3,True,False,1163.0,M. tuberculosis,Lineage 2,,96.283784,4736,0,1163,121,0,13970,CRyPTIC,v1.31,MDR,RRRRSSUSUSSSSSS,{'02-0918-22A153-1-14': 'b48e27b23f8377c209854...,/well/bag/jeffk/release_staging/,00/01/40/62/14062/site.02.iso.1.subject.0918.l...,dat/CRyPTIC2/V2/02/0918/22A153/1/regenotyped/,site.02.subj.0918.lab.22A153.iso.1.v0.8.3.rege...,,False,Lineage 2,lineage2.2.3,ERS5301016,site.02.iso.1.subject.0918.lab_id.22A153.seq_r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.10.subj.YA00023171.lab.YA00023171.iso.1,10,YA00023171,YA00023171,1,1,True,True,True,0.8.3,True,False,693.0,M. tuberculosis,Lineage 4,LAM,100.000000,5911,0,693,55,0,11874,CRyPTIC,v1.31,SUS,SSSSSSUSSSSSSSS,{'10-YA00023171-YA00023171-1-7': 'da79aa0564b0...,/well/bag/jeffk/release_staging/,00/00/31/24/3124/site.10.iso.1.subject.YA00023...,dat/CRyPTIC2/V2/10/YA00023171/YA00023171/1/reg...,site.10.subj.YA00023171.lab.YA00023171.iso.1.v...,,False,Lineage 4,lineage4.3.2.1,ERS5298216,site.10.iso.1.subject.YA00023171.lab_id.YA0002...
site.10.subj.SADH00634109_S16.lab.DH00634109_S16.iso.1,10,SADH00634109_S16,DH00634109_S16,1,1,False,True,False,0.8.3,True,True,NaN,,,,NaN,317,0,700,63,0,32,CRyPTIC,v1.31,SUS,SSSSSSSSSSSSSSS,,/well/bag/jeffk/release_staging/,0

In [ ]:
GENOMES.

Save UNIQUEIDs with inconclusive sequencing result in REMOVE list

In [4]:
EFFECTS = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/EFFECTS.pkl.gz')
EFFECTS.reset_index(inplace = True)
REMOVE = EFFECTS[EFFECTS.MUTATION.str[-1].isin(['O','X'])].UNIQUEID
REMOVE

8                     site.02.subj.0823.lab.2013241494.iso.1
9                     site.02.subj.0823.lab.2013241494.iso.1
24                        site.02.subj.0085.lab.22A036.iso.1
25                        site.02.subj.0085.lab.22A036.iso.1
26                    site.02.subj.0830.lab.2013221279.iso.1
                                 ...                        
1074535    site.10.subj.SATRL0073861_S19.lab.TRL0073861_S...
1074564         site.10.subj.YA00038960.lab.YA00038960.iso.1
1074565         site.10.subj.YA00038960.lab.YA00038960.iso.1
1074598         site.10.subj.YA00036589.lab.YA00036589.iso.1
1074599         site.10.subj.YA00036589.lab.YA00036589.iso.1
Name: UNIQUEID, Length: 158356, dtype: object

Make dataframe of all UNIQUEIDs that are classified as resistant

Include other resistances ()

In [5]:
EFFECTS = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/EFFECTS.pkl.gz')
EFFECTS.reset_index(inplace = True)
EFFECTS

,UNIQUEID,DRUG,GENE,MUTATION,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,SITEID,PREDICTION,DEFAULT_CATALOGUE
0,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoB,P45S,CRyPTIC,v1.31,GARC1,02,U,True
1,site.02.subj.0958.lab.22A197.iso.1,RIF,rpoB,P45S,CRyPTIC,v1.31,GARC1,02,U,True
2,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoB,S450L,CRyPTIC,v1.31,GARC1,02,U,True
3,site.02.subj.0958.lab.22A197.iso.1,RIF,rpoB,S450L,CRyPTIC,v1.31,GARC1,02,R,True
4,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoB,A1075A,CRyPTIC,v1.31,GARC1,02,S,True
...,...,...,...,...,...,...,...,...,...,...
1074679,site.10.subj.YA00044774.lab.YA00044774.iso.1,RFB,rpoC,A542A,CRyPTIC,v1.31,GARC1,10,S,True
1074680,site.10.subj.YA00166043.lab.YA00166043.iso.1,RIF,rpoB,D435V,CRyPTIC,v1.31,GARC1,10,R,True
1074681,site.10.subj.YA00166043.lab.YA00166043.iso.1,RFB,rpoB,D435V,CRyPTIC,v1.31,GARC1,10,U,True
1074682,site.10.subj.YA00166043.lab.YA00166043.iso.1,RIF,rpoB,A1075A,CRyPTIC,v1.31,GARC1,10,S,True


In [23]:
EFFECTS = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/EFFECTS.pkl.gz')
EFFECTS.reset_index(inplace = True)
EFFECTS['GENE_MUTATION'] = EFFECTS['GENE'] + '_' + EFFECTS['MUTATION']
EFFECTS = EFFECTS[(EFFECTS.DRUG=='RIF') & (EFFECTS.PREDICTION=='R') 
                  & (~EFFECTS.MUTATION.str[-1].isin(['O','X']))]

res_gene_mutations = EFFECTS.GENE_MUTATION
query = res_gene_mutations.unique()

MUTATIONS = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/MUTATIONS.pkl.gz')
MUTATIONS.reset_index(inplace = True)
MUTATIONS = MUTATIONS.astype({'GENE':'str'})
MUTATIONS['GENE_MUTATION'] = MUTATIONS['GENE'] + '_' + MUTATIONS['MUTATION']

resistant = MUTATIONS[MUTATIONS.GENE_MUTATION.isin(res_gene_mutations.unique())]
resistant.set_index('UNIQUEID', inplace=True)
resistant['IS_RES'] = True
resistant

,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION,IS_RES
UNIQUEID,,,,,,,,,,,,,,,,,,,,,,,,,,
site.02.subj.0958.lab.22A197.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_S450L,True
site.02.subj.0918.lab.22A153.iso.1,rpoB,D435G,435.0,435.0,NaN,NaN,gac,ggc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_D435G,True
site.02.subj.1033.lab.2013185075.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_S450L,True
site.02.subj.0739.lab.2013221518.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_S450L,True
site.02.subj.0104.lab.22A057.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_S450L,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.10.subj.SATRL0115669_S18.lab.TRL0115669_S18.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,10,1,rpoB_S450L,True
site.10.subj.XD01227882.lab.XD01227882.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,10,1,rpoB_S450L,True
site.10.subj.WG00269790.lab.WG00269790.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,10,1,rpoB_S450L,True


Make dataframe of all UNIQUEIDs that contain CMs

In [8]:
hits_final = pandas.read_csv('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/fisher_all.csv')
hits_final = hits_final[hits_final.both != 0]
hits_final = hits_final.iloc[:-1,:]

hits_final['log10'] = -numpy.log10(hits_final.p_value.astype('float'))

#create list of all hits above the 98% quantile of p-values
hits_final = hits_final[(hits_final.log10 > numpy.quantile(hits_final.log10, 0.98))]
print('There are', len(hits_final.other_mutation.unique()), 'hits above the indicated p-value quantile')

#remove all hits that are synonymous mutations or non-homoplastic
hits_final = hits_final[~(hits_final.other_mutation.apply(lambda x: x[5]==x[-1]))]
no_homoplasy = pandas.read_csv('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/all_hits_concat.csv')
no_homoplasy = no_homoplasy[no_homoplasy.homoplasy == False].CM
hits_final['homoplasy'] = (~hits_final.other_mutation.isin(no_homoplasy))
hits_final = hits_final[(hits_final.homoplasy == True)]

hits_final = hits_final.sort_values('both', ascending = False)
hits_final = hits_final.drop(['p_value', 'None', 'other', 'resistant', 'n_resistant', 'n_other', 'log10', 'homoplasy'], axis = 1)
hits_final.reset_index(drop = True, inplace = True)
hits_final

There are 96 hits above the indicated p-value quantile


,resistant_mutation,other_mutation,both
0,rpoB_S450L,rpoC_V483G,1206
1,rpoB_S450L,rpoC_I491V,665
2,rpoB_S450L,rpoC_V483A,586
3,rpoB_S450L,rpoC_I491T,457
4,rpoB_S450L,rpoC_P1040R,396
5,rpoB_S450L,rpoC_F452S,345
6,rpoB_S450L,rpoB_L731P,226
7,rpoB_S450L,rpoC_N698S,205
8,rpoB_S450L,rpoC_D485Y,194
9,rpoB_S450L,rpoC_V517L,184


In [9]:
compensatory = MUTATIONS[MUTATIONS.GENE_MUTATION.isin(hits_final.other_mutation.unique())]
compensatory.set_index('UNIQUEID', inplace=True)
compensatory['IS_CM'] = True
compensatory

,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION,IS_CM
UNIQUEID,,,,,,,,,,,,,,,,,,,,,,,,,,
site.02.subj.0958.lab.22A197.iso.1,rpoB,P45S,45.0,45.0,NaN,NaN,ccg,tcg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_P45S,True
site.02.subj.0918.lab.22A153.iso.1,rpoB,I491L,491.0,491.0,NaN,NaN,atc,ctc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoB_I491L,True
site.02.subj.0739.lab.2013221518.iso.1,rpoC,V483A,483.0,483.0,NaN,NaN,gtg,gcg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoC_V483A,True
site.02.subj.0885.lab.22A119.iso.1,rpoC,V483A,483.0,483.0,NaN,NaN,gtg,gcg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoC_V483A,True
site.02.subj.0411.lab.235087-14.iso.1,rpoA,T187A,187.0,187.0,NaN,NaN,acc,gcc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,rpoA_T187A,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.10.subj.SATRL0116843_S10.lab.TRL0116843_S10.iso.1,rpoC,V483G,483.0,483.0,NaN,NaN,gtg,ggg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,10,1,rpoC_V483G,True
site.10.subj.SATRL0116880_S12.lab.TRL0116880_S12.iso.1,rpoC,N698S,698.0,698.0,NaN,NaN,aac,agc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,10,1,rpoC_N698S,True
site.10.subj.SATRL0116187_S19.lab.TRL0116187_S19.iso.1,rpoC,V483G,483.0,483.0,NaN,NaN,gtg,ggg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,10,1,rpoC_V483G,True


Prevalence of CMs in resistant samples in lineage 2 (vs other lineages)

In [10]:
GENOMES.TB_TYPE_1.value_counts()

TB_TYPE_1
SUS    47914
MDR    16885
UNK     8900
XDR     2122
RIF     2039
Name: count, dtype: int64

In [11]:
res_CM = GENOMES[((GENOMES.TB_TYPE_1 == 'MDR') | (GENOMES.TB_TYPE_1 == 'XDR')) & (GENOMES.LINEAGE_NAME != 'Lineage 2') & (GENOMES.index.isin(compensatory.index))]
res = GENOMES[((GENOMES.TB_TYPE_1 == 'MDR') | (GENOMES.TB_TYPE_1 == 'XDR')) & (GENOMES.LINEAGE_NAME != 'Lineage 2')] #(GENOMES.TB_TYPE_1 == 'RIF') | 
print(len(res_CM), len(res), len(res_CM)/len(res))

5873 15756 0.37274689007362277


Calculate statistic for CM prevalence in resistant samples in Lineage 2 vs other Lineages:

In [12]:
data_res = [[2407, 3423], [11217, 17623]]
data_res = [[2059, 3423], [5980, 17623]]
data_MDR = [[2047, 3251], [5873, 15756]]

stat, p, dof, expected = chi2_contingency(data_MDR)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 1.2772599244343872e-60
Dependent (reject H0)


Now let's read in the mutational data for samples with specific mutations

In [13]:
MUTATIONS = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/MUTATIONS.pkl.gz')
MUTATIONS.reset_index(inplace=True)

rpob_s450l = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='S450L')]
rpob_s450l.set_index('UNIQUEID', inplace=True)
rpob_s450l['HAS_S450L'] = True
rpob_s450l

rpoc_e1092d = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='E1092D')] #No
rpoc_e1092d.set_index('UNIQUEID', inplace=True)
rpoc_e1092d['HAS_E1092D'] = True
rpoc_e1092d

rpoc_v483g = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='V483G')] #Yes
rpoc_v483g.set_index('UNIQUEID', inplace=True)
rpoc_v483g['HAS_V483G'] = True
rpoc_v483g

rpoc_i491v = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='I491V')] #Yes
rpoc_i491v.set_index('UNIQUEID', inplace=True)
rpoc_i491v['HAS_I491V'] = True
rpoc_i491v

rpoc_v483a = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='V483A')] #Yes
rpoc_v483a.set_index('UNIQUEID', inplace=True)
rpoc_v483a['HAS_V483A'] = True
rpoc_v483a

rpoc_i491t = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='I491T')] #Yes (but almost exclusively lineage 2)
rpoc_i491t.set_index('UNIQUEID', inplace=True)
rpoc_i491t['HAS_I491T'] = True
rpoc_i491t

rpoc_p1040r = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='P1040R')] #Yes
rpoc_p1040r.set_index('UNIQUEID', inplace=True)
rpoc_p1040r['HAS_P1040R'] = True
rpoc_p1040r

rpoc_g332s = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='G332S')] #Yes
rpoc_g332s.set_index('UNIQUEID', inplace=True)
rpoc_g332s['HAS_G332S'] = True
rpoc_g332s

rpoc_g433s = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='G433S')] #Yes
rpoc_g433s.set_index('UNIQUEID', inplace=True)
rpoc_g433s['HAS_G433S'] = True
rpoc_g433s


#rpoC_N698S
rpoc_n698s = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='N698S')] #Yes
rpoc_n698s.set_index('UNIQUEID', inplace=True)
rpoc_n698s['HAS_N698S'] = True
rpoc_n698s

#rpoC_F452S
rpoc_f452s = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='F452S')] #Yes
rpoc_f452s.set_index('UNIQUEID', inplace=True)
rpoc_f452s['HAS_F452S'] = True
rpoc_f452s

#rpoB_E761D
rpob_e761d = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='E761D')] #Yes
rpob_e761d.set_index('UNIQUEID', inplace=True)
rpob_e761d['HAS_E761D'] = True
rpob_e761d

#rpoB_L731P
rpob_l731p = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='L731P')] #Yes
rpob_l731p.set_index('UNIQUEID', inplace=True)
rpob_l731p['HAS_L731P'] = True
rpob_l731p

#rpoC_D485Y
rpoc_d485y = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='D485Y')] #Yes
rpoc_d485y.set_index('UNIQUEID', inplace=True)
rpoc_d485y['HAS_D485Y'] = True
rpoc_d485y

#rpoC_V517L
rpoc_v517l = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='V517L')] #Yes
rpoc_v517l.set_index('UNIQUEID', inplace=True)
rpoc_v517l['HAS_V517L'] = True
rpoc_v517l

#rpoC_V1252L
rpoc_v1252l = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='V1252L')] #Yes
rpoc_v1252l.set_index('UNIQUEID', inplace=True)
rpoc_v1252l['HAS_V1252L'] = True
rpoc_v1252l

#rpoA_T187A
rpoa_t187a = MUTATIONS[(MUTATIONS.GENE=='rpoA') & (MUTATIONS.MUTATION=='T187A')] #Yes
rpoa_t187a.set_index('UNIQUEID', inplace=True)
rpoa_t187a['HAS_T187A'] = True
rpoa_t187a


#rpoC_D485N
rpoc_d485n = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='D485N')] #Yes
rpoc_d485n.set_index('UNIQUEID', inplace=True)
rpoc_d485n['HAS_D485N'] = True
rpoc_d485n

#rpoC_L516P
rpoc_l516p = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='L516P')] #Yes
rpoc_l516p.set_index('UNIQUEID', inplace=True)
rpoc_l516p['HAS_L516P'] = True
rpoc_l516p

#rpoB_R827C
rpob_r827c = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='R827C')] #Yes
rpob_r827c.set_index('UNIQUEID', inplace=True)
rpob_r827c['HAS_R827C'] = True
rpob_r827c

#rpoC_P1040S
rpoc_p1040s = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='P1040S')] #Yes
rpoc_p1040s.set_index('UNIQUEID', inplace=True)
rpoc_p1040s['HAS_P1040S'] = True
rpoc_p1040s

#rpoC_G332R
rpoc_g332R = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='G332R')] #Yes
rpoc_g332R.set_index('UNIQUEID', inplace=True)
rpoc_g332R['HAS_G332R'] = True
rpoc_g332R

#rpoC_L527V
rpoc_l527v = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='L527V')] #Yes
rpoc_l527v.set_index('UNIQUEID', inplace=True)
rpoc_l527v['HAS_L527V'] = True
rpoc_l527v

#rpoC_P1040A
rpoc_p1040a = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='P1040A')] #Yes
rpoc_p1040a.set_index('UNIQUEID', inplace=True)
rpoc_p1040a['HAS_P1040A'] = True
rpoc_p1040a

#rpoB_I1106T
rpob_i1106t = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='I1106T')] #No
rpob_i1106t.set_index('UNIQUEID', inplace=True)
rpob_i1106t['HAS_I1106T'] = True
rpob_i1106t


#rpoC_K445R
rpoc_k445r = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='K445R')] #Yes
rpoc_k445r.set_index('UNIQUEID', inplace=True)
rpoc_k445r['HAS_K445R'] = True
rpoc_k445r

#rpoC_F452L
rpoc_f452l = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='F452L')] #Yes
rpoc_f452l.set_index('UNIQUEID', inplace=True)
rpoc_f452l['HAS_F452L'] = True
rpoc_f452l

#rpoC_L547V
rpoc_l547v = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='L547V')] #No
rpoc_l547v.set_index('UNIQUEID', inplace=True)
rpoc_l547v['HAS_L547V'] = True
rpoc_l547v

#rpoB_I480V
rpob_i480v = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='I480V')] #Yes
rpob_i480v.set_index('UNIQUEID', inplace=True)
rpob_i480v['HAS_I480V'] = True
rpob_i480v

#rpoB_K891E
rpob_k891e = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='K891E')] #Yes
rpob_k891e.set_index('UNIQUEID', inplace=True)
rpob_k891e['HAS_K891E'] = True
rpob_k891e

#rpoC_N416S
rpoc_n416s = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='N416S')] #Yes
rpoc_n416s.set_index('UNIQUEID', inplace=True)
rpoc_n416s['HAS_N416S'] = True
rpoc_n416s

#rpoC_E1033A
rpoc_e1033a = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='E1033A')] #Yes
rpoc_e1033a.set_index('UNIQUEID', inplace=True)
rpoc_e1033a['HAS_E1033A'] = True
rpoc_e1033a

#rpoC_S561P
rpoc_s561p = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='S561P')] #Yes
rpoc_s561p.set_index('UNIQUEID', inplace=True)
rpoc_s561p['HAS_S561P'] = True
rpoc_s561p


#rpoB_V168A
rpob_v168a = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='V168A')] #Yes
rpob_v168a.set_index('UNIQUEID', inplace=True)
rpob_v168a['HAS_V168A'] = True
rpob_v168a

#rpoB_H1028R
rpob_h1028r = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='H1028R')] #No
rpob_h1028r.set_index('UNIQUEID', inplace=True)
rpob_h1028r['HAS_H1028R'] = True
rpob_h1028r

#sigA_D146E
siga_d146e = MUTATIONS[(MUTATIONS.GENE=='sigA') & (MUTATIONS.MUTATION=='D146E')] #Yes
siga_d146e.set_index('UNIQUEID', inplace=True)
siga_d146e['HAS_D146E'] = True
siga_d146e

#rpoA_P25R
rpoa_p25r = MUTATIONS[(MUTATIONS.GENE=='rpoA') & (MUTATIONS.MUTATION=='P25R')] #Not enough data (1)
rpoa_p25r.set_index('UNIQUEID', inplace=True)
rpoa_p25r['HAS_P25R'] = True
rpoa_p25r

#rpoB_I491L
rpob_i491l = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='I491L')] #Yes
rpob_i491l.set_index('UNIQUEID', inplace=True)
rpob_i491l['HAS_I491L'] = True
rpob_i491l

#rpoC_G388A
rpoc_g388a = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='G388A')] #Yes
rpoc_g388a.set_index('UNIQUEID', inplace=True)
rpoc_g388a['HAS_G388A'] = True
rpoc_g388a

#sigA_A223T
siga_a223t = MUTATIONS[(MUTATIONS.GENE=='sigA') & (MUTATIONS.MUTATION=='A223T')] #No
siga_a223t.set_index('UNIQUEID', inplace=True)
siga_a223t['HAS_A223T'] = True
siga_a223t

#rpoB_E207K
rpob_e207k = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='E207K')] #No
rpob_e207k.set_index('UNIQUEID', inplace=True)
rpob_e207k['HAS_E207K'] = True
rpob_e207k


#rpoB_R167C
rpob_r167c = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='R167C')] #Yes
rpob_r167c.set_index('UNIQUEID', inplace=True)
rpob_r167c['HAS_R167C'] = True
rpob_r167c

#rpoC_G571R
rpoc_g571r = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='G571R')] #Yes
rpoc_g571r.set_index('UNIQUEID', inplace=True)
rpoc_g571r['HAS_G571R'] = True
rpoc_g571r

#rpoC_L405M
rpoc_l405m = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='L405M')] #No data (0)
rpoc_l405m.set_index('UNIQUEID', inplace=True)
rpoc_l405m['HAS_L405M'] = True
rpoc_l405m

#rpoB_L464M
rpob_l464m = MUTATIONS[(MUTATIONS.GENE=='rpoB') & (MUTATIONS.MUTATION=='L464M')] #No data (0)
rpob_l464m.set_index('UNIQUEID', inplace=True)
rpob_l464m['HAS_L464M'] = True
rpob_l464m

#rpoZ_T107I
rpoz_t107i = MUTATIONS[(MUTATIONS.GENE=='rpoZ') & (MUTATIONS.MUTATION=='T107I')] #No data (0)
rpoz_t107i.set_index('UNIQUEID', inplace=True)
rpoz_t107i['HAS_T107I'] = True
rpoz_t107i

#rpoC_T853A
rpoc_t853a = MUTATIONS[(MUTATIONS.GENE=='rpoC') & (MUTATIONS.MUTATION=='T853A')] #Yes
rpoc_t853a.set_index('UNIQUEID', inplace=True)
rpoc_t853a['HAS_T853A'] = True
rpoc_t853a

#make process faster (list of mutations that are missing, loop through them and make dataframe)
mutation_list = ['rpoC_G594E','rpoB_c-61t','rpoB_V695L','rpoC_W484G','rpoB_A692T','rpoA_V183G','rpoC_P481T','rpoB_Q409R','rpoC_V1039A','rpoC_A521D','rpoB_Q975H','rpoB_P45S','rpoC_N826T','rpoA_-40_indel','rpoC_L507V','rpoC_V431M','rpoB_V496A','rpoB_I488V','rpoC_V1252M','rpoB_A286V','rpoC_K1152Q','rpoC_T812I','rpoC_G519D','rpoC_L449V','rpoB_R827L','rpoC_F452C','rpoB_A405P','rpoA_A180V','rpoC_N698K','rpoA_D190G','rpoC_H525Q','rpoC_P434R','rpoA_G31S','rpoB_P45L','sigA_247_indel','rpoA_E184D','rpoB_F503S','rpoB_T400A','rpoC_S428A','rpoB_I491V','rpoB_S874Y','rpoB_R552L','rpoA_G31A','rpoC_D747A','rpoC_P1040L','rpoC_E1033K','rpoC_R741S','rpoC_E1137G','rpoC_T1230I','rpoC_W484S','rpoB_H835R','rpoC_L449R','rpoC_N698H','rpoB_R827H','rpoA_R182L','rpoC_F831L','rpoZ_P85S','rpoC_G332C','rpoB_V496M','rpoC_H525N','rpoB_V534M','rpoC_Q523E','rpoC_E518D','rpoB_R552H','rpoC_P1040Q','rpoB_E550G','rpoB_T399I','rpoC_Q1125H','rpoB_F971L','rpoC_R770H','rpoC_P434Q','rpoC_P434T','rpoA_V183A','rpoC_V1039G','rpoC_K1152N','rpoC_P1040T','rpoC_A734V','rpoC_G433A','rpoC_K715T','rpoC_E750G','rpoB_H723D','rpoC_I885V','rpoC_E750D','rpoC_Q1110H','rpoB_L42V','rpoC_G433C','rpoC_E1113G','rpoB_I588V','rpoC_E1140D','rpoC_E757G','rpoB_V970A','rpoC_Q479R','rpoB_V534A','rpoB_Y564H','rpoB_L554P','rpoB_R552C','rpoC_P434A','rpoB_D574E','rpoA_T181A','rpoC_A492V','rpoB_I873F','rpoC_G519R','rpoA_T187P','rpoC_E488Q','rpoB_T400S','rpoC_D747G','rpoC_A1213E','rpoB_a-83g','rpoC_V1124G','rpoC_N416T','rpoB_P45T','rpoC_W484L','rpoC_V1147A','rpoC_D735E','rpoC_H748P','rpoB_N487S','rpoB_I491M','rpoB_L862R']

In [15]:
#make process faster (list of mutations that are missing, loop through them and make dataframe)
#mutation_list = ['rpoC_G594E','rpoB_c-61t','rpoB_V695L','rpoC_W484G','rpoB_A692T','rpoA_V183G','rpoC_P481T','rpoB_Q409R','rpoC_V1039A','rpoC_A521D','rpoB_Q975H','rpoB_P45S','rpoC_N826T','rpoA_-40_indel','rpoC_L507V','rpoC_V431M','rpoB_V496A','rpoB_I488V','rpoC_V1252M','rpoB_A286V','rpoC_K1152Q','rpoC_T812I','rpoC_G519D','rpoC_L449V','rpoB_R827L','rpoC_F452C','rpoB_A405P','rpoA_A180V','rpoC_N698K','rpoA_D190G','rpoC_H525Q','rpoC_P434R','rpoA_G31S','rpoB_P45L','sigA_247_indel','rpoA_E184D','rpoB_F503S','rpoB_T400A','rpoC_S428A','rpoB_I491V','rpoB_S874Y','rpoB_R552L','rpoA_G31A','rpoC_D747A','rpoC_P1040L','rpoC_E1033K','rpoC_R741S','rpoC_E1137G','rpoC_T1230I','rpoC_W484S','rpoB_H835R','rpoC_L449R','rpoC_N698H','rpoB_R827H','rpoA_R182L','rpoC_F831L','rpoZ_P85S','rpoC_G332C','rpoB_V496M','rpoC_H525N','rpoB_V534M','rpoC_Q523E','rpoC_E518D','rpoB_R552H','rpoC_P1040Q','rpoB_E550G','rpoB_T399I','rpoC_Q1125H','rpoB_F971L','rpoC_R770H','rpoC_P434Q','rpoC_P434T','rpoA_V183A','rpoC_V1039G','rpoC_K1152N','rpoC_P1040T','rpoC_A734V','rpoC_G433A','rpoC_K715T','rpoC_E750G','rpoB_H723D','rpoC_I885V','rpoC_E750D','rpoC_Q1110H','rpoB_L42V','rpoC_G433C','rpoC_E1113G','rpoB_I588V','rpoC_E1140D','rpoC_E757G','rpoB_V970A','rpoC_Q479R','rpoB_V534A','rpoB_Y564H','rpoB_L554P','rpoB_R552C','rpoC_P434A','rpoB_D574E','rpoA_T181A','rpoC_A492V','rpoB_I873F','rpoC_G519R','rpoA_T187P','rpoC_E488Q','rpoB_T400S','rpoC_D747G','rpoC_A1213E','rpoB_a-83g','rpoC_V1124G','rpoC_N416T','rpoB_P45T','rpoC_W484L','rpoC_V1147A','rpoC_D735E','rpoC_H748P','rpoB_N487S','rpoB_I491M','rpoB_L862R']
#mutation_list = ['rpoB_A1075A','rpoB_A451A','rpoZ_S22S','rpoC_S548S','rpoB_G453G','rpoB_S431S','rpoC_C62C','rpoB_Q432Q','rpoB_G442G','rpoC_D404D','rpoB_L443L','rpoC_A542A','rpoB_S428S','rpoB_G426G','rpoB_P416P','sigA_V375V']
mutation_list = ['sigA_L386M','sigA_G380A','rpoC_I128V','sigA_I382V','rpoB_R791T','rpoB_E460D','sigA_E385Q']
mutation_list

data_frames = [GENOMES[['ORIGINAL_UID', 'LINEAGE_NAME']]]

d = {}
for i,name in enumerate(mutation_list):
    d["{0}".format(name)] = MUTATIONS[(MUTATIONS.GENE==name[0:4]) & (MUTATIONS.MUTATION==name[5:])]
    d["{0}".format(name)].set_index('UNIQUEID', inplace=True)
    d["{0}".format(name)]['HAS_MUTATION'] = True
    
d.keys()

dict_keys(['sigA_L386M', 'sigA_G380A', 'rpoC_I128V', 'sigA_I382V', 'rpoB_R791T', 'rpoB_E460D', 'sigA_E385Q'])

In [16]:
LABELS = GENOMES[['ORIGINAL_UID', 'LINEAGE_NAME']].join(rpob_s450l[['HAS_S450L']], how='left')
LABELS.fillna(value={'HAS_S450L': False}, inplace=True)

for i,name in enumerate(d.keys()):
    LABELS = LABELS.join(d[name]['HAS_MUTATION'], how='left')
    LABELS.rename(columns=({ 'HAS_MUTATION': name}), inplace=True)
    LABELS.fillna(value={name: False}, inplace=True)

LABELS.reset_index(inplace=True)
LABELS.set_index('ORIGINAL_UID', inplace=True)
LABELS

,UNIQUEID,LINEAGE_NAME,HAS_S450L,sigA_L386M,sigA_G380A,rpoC_I128V,sigA_I382V,rpoB_R791T,rpoB_E460D,sigA_E385Q
ORIGINAL_UID,,,,,,,,,,
site.02.iso.1.subject.0958.lab_id.22A197.seq_reps.197,site.02.subj.0958.lab.22A197.iso.1,Lineage 2,True,False,False,False,False,False,False,False
site.02.iso.1.subject.0823.lab_id.2013241494.seq_reps.241494,site.02.subj.0823.lab.2013241494.iso.1,Lineage 4,False,False,False,False,False,False,False,False
site.02.iso.1.subject.0359.lab_id.222018-14.seq_reps.14222018,site.02.subj.0359.lab.222018-14.iso.1,Lineage 2,False,False,False,False,False,False,False,False
site.02.iso.1.subject.0224.lab_id.2013221088.seq_reps.13221088_2013221088,site.02.subj.0224.lab.2013221088.iso.1,Lineage 2,False,False,False,False,False,False,False,False
site.02.iso.1.subject.0918.lab_id.22A153.seq_reps.153,site.02.subj.0918.lab.22A153.iso.1,Lineage 2,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
site.10.iso.1.subject.YA00023171.lab_id.YA00023171.seq_reps.1,site.10.subj.YA00023171.lab.YA00023171.iso.1,Lineage 4,False,False,False,False,False,False,False,False
None,site.10.subj.SADH00634109_S16.lab.DH00634109_S...,,False,False,False,False,False,False,False,False
None,site.10.subj.H37RV_SA2437.lab.H37RV_SA2437.iso.1,,False,False,False,False,False,False,False,False


Join the boolean dataframes with the LABEL dataframe

In [17]:
LABELS = GENOMES[['ORIGINAL_UID', 'LINEAGE_NAME']].join(rpob_s450l[['HAS_S450L']], how='left')
LABELS.fillna(value={'HAS_S450L': False}, inplace=True)
LABELS.reset_index(inplace=True)
LABELS.set_index('ORIGINAL_UID', inplace=True)
LABELS.index.value_counts()

ORIGINAL_UID
site.02.iso.1.subject.0958.lab_id.22A197.seq_reps.197              1
site.13.iso.1.subject.180200018.lab_id.180200018.seq_reps.1        1
site.13.iso.1.subject.140300022.lab_id.140300022.seq_reps.1        1
site.13.iso.1.subject.080200106.lab_id.080200106.seq_reps.1        1
site.13.iso.1.subject.180200089.lab_id.180200089.seq_reps.1        1
                                                                  ..
site.11.iso.1.subject.XTB_17-589.lab_id.XTB_17-589.seq_reps.1      1
site.11.iso.1.subject.XTB_18-022.lab_id.XTB_18-022.seq_reps.1_2    1
site.11.iso.1.subject.XTB_17-188.lab_id.XTB_17-188.seq_reps.1      1
site.11.iso.1.subject.XTB_18-108.lab_id.XTB_18-108.seq_reps.1      1
site.10.iso.1.subject.YA00166043.lab_id.YA00166043.seq_reps.1      1
Name: count, Length: 15228, dtype: int64

Tried more efficient way of joining multiple dataframes (still has same issue of duplicated UNIQUEIDs when joining resistant and compensatory)

In [13]:
data_frames = [GENOMES[['ORIGINAL_UID', 'LINEAGE_NAME']], resistant[['IS_RES']], compensatory[['IS_CM']],
               rpoc_e1092d[['HAS_E1092D']], rpoc_v483g[['HAS_V483G']], rpob_s450l[['HAS_S450L']], 
               rpoc_i491v[['HAS_I491V']], rpoc_v483a[['HAS_V483A']], rpoc_i491t[['HAS_I491T']], 
               rpoc_p1040r[['HAS_P1040R']],rpoc_g332s[['HAS_G332S']], rpoc_g433s[['HAS_G433S']], 
               rpoc_n698s[['HAS_N698S']], rpoc_f452s[['HAS_F452S']], rpob_e761d[['HAS_E761D']], 
               rpob_l731p[['HAS_L731P']], rpoc_d485y[['HAS_D485Y']], rpoc_v517l[['HAS_V517L']], 
               rpoc_v1252l[['HAS_V1252L']], rpoa_t187a[['HAS_T187A']], rpoc_d485n[['HAS_D485N']], 
               rpoc_l516p[['HAS_L516P']], rpob_r827c[['HAS_R827C']], rpoc_p1040s[['HAS_P1040S']], 
               rpoc_g332R[['HAS_G332R']], rpoc_l527v[['HAS_L527V']], rpoc_p1040a[['HAS_P1040A']], 
               rpob_i1106t[['HAS_I1106T']], rpoc_k445r[['HAS_K445R']], rpoc_f452l[['HAS_F452L']], 
               rpoc_l547v[['HAS_L547V']], rpob_i480v[['HAS_I480V']], rpob_k891e[['HAS_K891E']], 
               rpoc_n416s[['HAS_N416S']], rpoc_e1033a[['HAS_E1033A']], rpoc_s561p[['HAS_S561P']], 
               rpob_v168a[['HAS_V168A']], rpob_h1028r[['HAS_H1028R']], siga_d146e[['HAS_D146E']], 
               rpoa_p25r[['HAS_P25R']], rpob_i491l[['HAS_I491L']], rpoc_g388a[['HAS_G388A']], 
               siga_a223t[['HAS_A223T']], rpob_e207k[['HAS_E207K']], rpob_r167c[['HAS_R167C']], 
               rpoc_g571r[['HAS_G571R']], rpoc_l405m[['HAS_L405M']], rpob_l464m[['HAS_L464M']], 
               rpoz_t107i[['HAS_T107I']], rpoc_t853a[['HAS_T853A']]]

LABELS = reduce(lambda  left,right: pandas.merge(left,right, on=['UNIQUEID'],
                                            how='outer'), data_frames)
LABELS.replace(numpy.nan, False, inplace = True)
LABELS.reset_index(inplace=True)
LABELS.set_index('ORIGINAL_UID', inplace=True)
LABELS.index.value_counts()

ORIGINAL_UID
False                                                                    63079
site.03.iso.1.subject.JPN-R2012-0044.lab_id.JPN-R2012-0044.seq_reps.1        3
site.02.iso.1.subject.0183.lab_id.2013222280.seq_reps.2013222280             3
site.02.iso.1.subject.0070.lab_id.22A021.seq_reps.21                         3
site.02.iso.1.subject.0133.lab_id.22A088.seq_reps.88                         3
                                                                         ...  
site.11.iso.1.subject.XTB-18-189.lab_id.XTB-18-189.seq_reps.1                1
site.11.iso.1.subject.XTB_18-028.lab_id.XTB_18-028.seq_reps.1_2              1
site.11.iso.1.subject.XTB-18-160.lab_id.XTB-18-160.seq_reps.1                1
site.11.iso.1.subject.XTB_17-505.lab_id.XTB_17-505.seq_reps.1_2              1
site.10.iso.1.subject.YA00166043.lab_id.YA00166043.seq_reps.1                1
Name: count, Length: 15229, dtype: int64

In [18]:
LABELS

,UNIQUEID,LINEAGE_NAME,HAS_S450L
ORIGINAL_UID,,,
site.02.iso.1.subject.0958.lab_id.22A197.seq_reps.197,site.02.subj.0958.lab.22A197.iso.1,Lineage 2,True
site.02.iso.1.subject.0823.lab_id.2013241494.seq_reps.241494,site.02.subj.0823.lab.2013241494.iso.1,Lineage 4,False
site.02.iso.1.subject.0359.lab_id.222018-14.seq_reps.14222018,site.02.subj.0359.lab.222018-14.iso.1,Lineage 2,False
site.02.iso.1.subject.0224.lab_id.2013221088.seq_reps.13221088_2013221088,site.02.subj.0224.lab.2013221088.iso.1,Lineage 2,False
site.02.iso.1.subject.0918.lab_id.22A153.seq_reps.153,site.02.subj.0918.lab.22A153.iso.1,Lineage 2,False
...,...,...,...
site.10.iso.1.subject.YA00023171.lab_id.YA00023171.seq_reps.1,site.10.subj.YA00023171.lab.YA00023171.iso.1,Lineage 4,False
None,site.10.subj.SADH00634109_S16.lab.DH00634109_S...,,False
None,site.10.subj.H37RV_SA2437.lab.H37RV_SA2437.iso.1,,False


Read in the Newick file downloaded from ITOL. To get this I uploaded the Newick file in the `Brankin_Malone` repo and then exported it again as a Newick. Don't know why but trying to use the original Newick leads to errors in `tsconvert` lower down

In [19]:
with open('cryptic_tree.itol.newick', 'r') as f:
    cryptic_newick = f.read()

print(cryptic_newick[:1000])

(site.04.iso.1.subject.03818.lab_id.830476.seq_reps.1:6.90800,site.04.iso.1.subject.00310.lab_id.701951.seq_reps.1:3.09200,(site.04.iso.1.subject.01627.lab_id.27972.seq_reps.1:0.46454,(site.04.iso.1.subject.01042.lab_id.717652.seq_reps.1:8.18534,(((site.04.iso.1.subject.02112.lab_id.803805.seq_reps.1:9.91015,site.04.iso.1.subject.04137.lab_id.832790.seq_reps.1:8.08985):0.17857,((((site.04.iso.1.subject.01432.lab_id.724551.seq_reps.1:1.73206,site.04.iso.1.subject.02235.lab_id.805177.seq_reps.1:4.26794):0.87800,(site.04.iso.1.subject.00300.lab_id.702487.seq_reps.1:0.00000,site.04.iso.1.subject.03648.lab_id.JJH9682.seq_reps.1:2.00000):3.62200):8.94817,(site.04.iso.1.subject.00564.lab_id.709046.seq_reps.1:7.96229,(site.04.iso.1.subject.04356.lab_id.903933.seq_reps.1:5.42753,(site.04.iso.1.subject.01175.lab_id.721997.seq_reps.1:1.00000,site.04.iso.1.subject.01288.lab_id.801278.seq_reps.1:0.00000):3.07247):2.53771):1.48933):0.11168,(((site.04.iso.1.subject.02152.lab_id.803055.seq_reps.1:0.89

Use `tsconvert` to convert to a succint tree sequence

In [20]:
ts = tsconvert.from_newick(cryptic_newick, min_edge_length=0.001)
print('done')

done


Get a copy of the underlying tables that describe the tree so we can relate the `node id` to the `UNIQUEID`

In [21]:
new_tables = ts.dump_tables()  # make a copy of the tree sequence tables, for parsing
new_tables.nodes[:3]

id,flags,population,individual,time,metadata
0,0,-1,-1,2060.10904000,{}
1,1,-1,-1,2053.20104000,{'name': 'site.04.iso.1.subject.03818...
2,1,-1,-1,2057.01704000,{'name': 'site.04.iso.1.subject.00310...


Attempt to prune the tree by removing lineages that are not 1-4, and removing outliers/ inconclusive sequences

In [31]:
vis = 'prune'

prune_header="""PRUNE

DATA
"""

prune_annotations = ''

# LABELS = GENOMES[['ORIGINAL_UID', 'LINEAGE_NAME']]

for i in tqdm(range(ts.num_nodes)):  

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        lineage = LABELS.loc[uid]['LINEAGE_NAME']
        
        if i == 15072 or i == 15074 or i == 15077 or i == 15076:
            print('outliers:', row.metadata)
            
            #uncomment to exclude outliers:
            #continue
            
        if LABELS.loc[uid].UNIQUEID in REMOVE.to_list():
            print('inconclusive sequencing:', str(i), row.metadata)
            
            #uncomment to exclude inconclusive sequences:
            continue
        
        if lineage!='' and lineage[0]=='L':
            
            if lineage == 'Lineage 1' or lineage == 'Lineage 2' or lineage == 'Lineage 3' or lineage == 'Lineage 4':
                line = 'n' + str(i) + '\n'

                prune_annotations += line 

            else:
                print('other lineage:', i)
                
                continue
                
        else:
            print('no lineage:', i)
            lineage = None

prune_annotations = prune_header + prune_annotations

  0%|                                       | 51/30454 [00:00<01:59, 253.68it/s]

inconclusive sequencing: 1 {'name': 'site.04.iso.1.subject.03818.lab_id.830476.seq_reps.1'}


  0%|▏                                     | 150/30454 [00:00<01:38, 309.00it/s]

inconclusive sequencing: 106 {'name': 'site.04.iso.1.subject.03595.lab_id.819090.seq_reps.1'}
inconclusive sequencing: 139 {'name': 'site.04.iso.1.subject.04511.lab_id.905563.seq_reps.1_2'}
inconclusive sequencing: 164 {'name': 'site.04.iso.1.subject.04554.lab_id.903942.seq_reps.1'}


  1%|▎                                     | 252/30454 [00:00<01:33, 322.95it/s]

...

 99%|███████████████████████████████████▊| 30286/30454 [01:47<00:00, 342.59it/s]

inconclusive sequencing: 30220 {'name': 'site.05.iso.1.subject.LR-2426.lab_id.FN-01116-15.seq_reps.1'}
inconclusive sequencing: 30222 {'name': 'site.05.iso.1.subject.PTAN-0100.lab_id.TAN-306.seq_reps.1'}
inconclusive sequencing: 30228 {'name': 'site.05.iso.1.subject.PMOP-0526.lab_id.MOP-044.seq_reps.1'}
inconclusive sequencing: 30230 {'name': 'site.05.iso.1.subject.LR-2353.lab_id.FN-00515-16.seq_reps.1'}


100%|███████████████████████████████████▉| 30391/30454 [01:48<00:00, 335.55it/s]

inconclusive sequencing: 30344 {'name': 'site.05.iso.1.subject.CA-1360.lab_id.CO-02151-19.seq_reps.1'}
inconclusive sequencing: 30352 {'name': 'site.05.iso.1.subject.PSLM-0843.lab_id.SLM-112.seq_reps.1'}


100%|████████████████████████████████████| 30454/30454 [01:48<00:00, 281.01it/s]


Dynamically create the ITOL annotations file to add symbols at the end of the branches coloured by lineage

In [282]:
vis = 'symbol'

lineage_header="""DATASET_SYMBOL

SEPARATOR COMMA

DATASET_LABEL,example symbols

COLOR,#e41a1c

MAXIMUM_SIZE,3


DATA
"""

colour_lookup = {1: '#4daf4a', 2: '#e41a1c', 3: '#984ea3', 4: '#377eb8', 5: '#ff7f00', 6: '#a65628'}

lineage_annotations = ''

for i in tqdm(range(ts.num_nodes)):
# for i in tqdm(range(20010,20040)):    

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        lineage = LABELS.loc[uid]['LINEAGE_NAME']
        if lineage!='' and lineage[0]=='L':
            lineage=int(lineage[-1])
        else:
            lineage=None

        if lineage in colour_lookup.keys():
            colour = colour_lookup[lineage]
        else:
            colour = '#bbbbbb'

        line = 'n'+str(i) +',1,1,' + colour + ',1,1\n'
        lineage_annotations += line 
    

lineage_annotations = lineage_header + lineage_annotations
            

100%|███████████████████████████████████| 30454/30454 [00:24<00:00, 1263.32it/s]


Display lineage as colored strip instead

In [153]:
vis = 'colorstrip'

lineage_header="""DATASET_COLORSTRIP

SEPARATOR COMMA

DATASET_LABEL,label1

COLOR,#e41a1c

COLOR_BRANCHES,0


DATA
"""

colour_lookup = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a', 4: '#ff7f00', 5: '#ff7f00', 6: '#a65628'}

lineage_annotations = ''

for i in tqdm(range(ts.num_nodes)):
# for i in tqdm(range(20010,20040)):    

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        lineage = LABELS.loc[uid]['LINEAGE_NAME']
        if lineage!='' and lineage[0]=='L':
            lineage=int(lineage[-1])
        else:
            lineage=None

        if lineage in colour_lookup.keys():
            colour = colour_lookup[lineage]
        else:
            colour = '#bbbbbb'

        line = 'n'+ str(i) + ',' + colour + ',Lineage ' + str(lineage) + '\n'
        lineage_annotations += line 
    

lineage_annotations = lineage_header + lineage_annotations


100%|███████████████████████████████████| 30454/30454 [00:27<00:00, 1093.67it/s]


Display lineages as colored ranges

In [20]:
vis = 'colorstrip'

lineage_header="""TREE_COLORS

SEPARATOR COMMA

DATA
"""

colour_lookup = {1: '#4daf4a', 2: '#e41a1c', 3: '#984ea3', 4: '#377eb8', 5: '#ff7f00', 6: '#a65628'}

lineage_annotations = ''

for i in tqdm(range(ts.num_nodes)):  

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        lineage = LABELS.loc[uid]['LINEAGE_NAME']
        if lineage!='' and lineage[0]=='L':
            lineage=int(lineage[-1])
        else:
            lineage=None

        if lineage in colour_lookup.keys():
            colour = colour_lookup[lineage]
        else:
            colour = '#bbbbbb'

        line = 'n'+ str(i) + ',' + 'range,' + colour + ',Lineage ' + str(lineage) + '\n'
        lineage_annotations += line 
    

lineage_annotations = lineage_header + lineage_annotations


100%|███████████████████████████████████| 30454/30454 [00:25<00:00, 1208.66it/s]


Some ORIGINAL_IDS are duplicated due to a step before (join LINEAGE), but it would make no difference to simply choose one of the instances

In [24]:
list = []
for i in tqdm(range(ts.num_nodes)):  
    row = new_tables.nodes[i]
    if 'name' in row.metadata.keys():
        uid = row.metadata['name']
        mutation = LABELS.loc[uid]
        if mutation.IS_RES.size > 1:  
            print(mutation.IS_RES.size, mutation.index)
            list.append(mutation.iloc[0].IS_RES == mutation.iloc[1].IS_RES)
print(all(list))

  5%|█▉                                  | 1662/30454 [00:01<00:22, 1272.41it/s]

2 Index(['site.17.iso.1.subject.K0057.lab_id.123-20-0057-1000.seq_reps.1', 'site.17.iso.1.subject.K0057.lab_id.123-20-0057-1000.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.08.iso.1.subject.02TB2346.lab_id.26277.seq_reps.1', 'site.08.iso.1.subject.02TB2346.lab_id.26277.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.08.iso.1.subject.24TB00-016.lab_id.2394.seq_reps.1', 'site.08.iso.1.subject.24TB00-016.lab_id.2394.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.08.iso.1.subject.24TB00-016.lab_id.2297.seq_reps.1', 'site.08.iso.1.subject.24TB00-016.lab_id.2297.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.08.iso.1.subject.24TB00-016.lab_id.2248.seq_reps.1', 'site.08.iso.1.subject.24TB00-016.lab_id.2248.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.08.iso.1.subject.24TB00-016.lab_id.2296.seq_reps.1', 'site.08.iso.1.subject.24TB00-016.lab_id.2296.seq_reps.1'], dtype='object', name='ORIGINAL_UID'

  7%|██▌                                 | 2167/30454 [00:01<00:24, 1132.70it/s]

2 Index(['site.02.iso.1.subject.0129.lab_id.22A084.seq_reps.84', 'site.02.iso.1.subject.0129.lab_id.22A084.seq_reps.84'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.02.iso.1.subject.0486.lab_id.235060-14.seq_reps.14235060', 'site.02.iso.1.subject.0486.lab_id.235060-14.seq_reps.14235060'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.02.iso.1.subject.0114.lab_id.22A068.seq_reps.68', 'site.02.iso.1.subject.0114.lab_id.22A068.seq_reps.68'], dtype='object', name='ORIGINAL_UID')


  8%|███                                 | 2559/30454 [00:02<00:22, 1237.15it/s]

...

2 Index(['site.10.iso.1.subject.KD01666167.lab_id.KD01666167.seq_reps.1', 'site.10.iso.1.subject.KD01666167.lab_id.KD01666167.seq_reps.1'], dtype='object', name='ORIGINAL_UID')


 98%|██████████████████████████████████▏| 29730/30454 [00:24<00:00, 1196.85it/s]

2 Index(['site.05.iso.1.subject.PMK-1048.lab_id.MK-2348.seq_reps.1', 'site.05.iso.1.subject.PMK-1048.lab_id.MK-2348.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.10.iso.1.subject.BA00346437.lab_id.BA00346437.seq_reps.1', 'site.10.iso.1.subject.BA00346437.lab_id.BA00346437.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.10.iso.1.subject.YA00013886.lab_id.YA00013886.seq_reps.1', 'site.10.iso.1.subject.YA00013886.lab_id.YA00013886.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.17.iso.1.subject.K0059.lab_id.123-20-0059-1000.seq_reps.1', 'site.17.iso.1.subject.K0059.lab_id.123-20-0059-1000.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.10.iso.1.subject.YA00038859.lab_id.YA00038859.seq_reps.1', 'site.10.iso.1.subject.YA00038859.lab_id.YA00038859.seq_reps.1'], dtype='object', name='ORIGINAL_UID')
2 Index(['site.17.iso.1.subject.K0091.lab_id.123-20-0091-1000.seq_reps.1', 'site.17.iso.1.subject.K0091.lab_id.123-20-0091-

100%|███████████████████████████████████| 30454/30454 [00:24<00:00, 1224.16it/s]

True


Display resistance/ CM presence on nodes (red = resistance mutation, blue = CM, green = both)

In [28]:
vis = 'symbol'

mutation_header="""DATASET_SYMBOL

SEPARATOR COMMA

DATASET_LABEL,example symbols

COLOR,#e41a1c

MAXIMUM_SIZE,3


DATA
"""

colour_lookup = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a'}

mutation_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        mutation = LABELS.loc[uid]
        
        if mutation.IS_RES.size > 1:
            mutation = mutation.iloc[0]
            
        if mutation.IS_RES == True:
            if mutation.IS_CM == True:
                mutation = 3
            else:
                mutation = 1
        else:
            if mutation.IS_CM == True:
                mutation = 2
            else:
                mutation = None

        if mutation in colour_lookup.keys():
            colour = colour_lookup[mutation]
            
            line = 'n'+str(i) +',1,1,' + colour + ',1,0.5\n'
            mutation_annotations += line 
        else:
            continue
    

mutation_annotations = mutation_header + mutation_annotations


100%|███████████████████████████████████| 30454/30454 [00:22<00:00, 1340.57it/s]


Do the same for specific combinations of CM + S450L

In [47]:
vis = 'symbol'

mutation_header="""DATASET_SYMBOL

SEPARATOR COMMA

DATASET_LABEL,example symbols

COLOR,#e41a1c

MAXIMUM_SIZE,3


DATA
"""

colour_lookup = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a'}

mutation_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        mutation = LABELS.loc[uid]
        
        if mutation.IS_RES.size > 1:
            mutation = mutation.iloc[0]
            
        if mutation.HAS_S450L == True:
            if mutation.HAS_E1092D == True:
                mutation = 3
            else:
                mutation = 1
        else:
            if mutation.HAS_E1092D == True:
                mutation = 2
            else:
                mutation = None

        if mutation in colour_lookup.keys():
            colour = colour_lookup[mutation]
            
            line = 'n'+str(i) +',1,1,' + colour + ',1,0.5\n'
            mutation_annotations += line 
        else:
            continue
    

mutation_annotations = mutation_header + mutation_annotations


100%|███████████████████████████████████| 30454/30454 [00:22<00:00, 1356.06it/s]


Do the same for single CMs without resistance mutations

In [38]:
vis = 'symbol'

mutation_header="""DATASET_SYMBOL

SEPARATOR COMMA

DATASET_LABEL,example symbols

COLOR,#e41a1c

MAXIMUM_SIZE,3


DATA
"""

colour_lookup = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a', 4: '#984ea3', 5: '#ff7f00', 6: '#a65628'}

mutation_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        mutation = LABELS.loc[uid]
        
        if mutation.IS_RES.size > 1:
            mutation = mutation.iloc[0]
            
        if mutation.HAS_V483G == True or mutation.HAS_V483A == True:
            mutation = 1

        elif mutation.HAS_I491V == True or mutation.HAS_I491T == True:
            mutation = 2
            
        elif mutation.HAS_P1040R == True:
            mutation = 3
            
        elif mutation.HAS_G332S == True:
            mutation = 4
            
        elif mutation.HAS_G433S == True:
            mutation = 5
            
        elif mutation.HAS_N698S == True:
            mutation = 6
            
        else:
            mutation = None

        if mutation in colour_lookup.keys():
            colour = colour_lookup[mutation]
            
            line = 'n'+str(i) +',1,1,' + colour + ',1,0.5\n'
            mutation_annotations += line 
        else:
            continue
    

mutation_annotations = mutation_header + mutation_annotations


100%|███████████████████████████████████| 30454/30454 [00:27<00:00, 1113.62it/s]


Dynamically create the ITOL annotations file to small circles around the circle if the sample contains an S450L mutation in rpoB

In [100]:
vis = 'binary'

binary_header="""DATASET_BINARY

SEPARATOR COMMA

DATASET_LABEL,binary

COLOR,#e41a1c

FIELD_SHAPES,1

FIELD_LABELS,f1

DATA
"""

binary_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']

        if uid in LABELS[LABELS['HAS_T853A']].index:
            line = 'n'+str(i) +',1\n'
            binary_annotations += line 

binary_annotations = binary_header + binary_annotations

with open('ct-CM-annotations_name_circle.txt','w') as f:
    f.write(binary_annotations)

  0%|                                       | 1/30454 [00:00<00:08, 3799.19it/s]


KeyError: 'HAS_T853A'

for new hits through fishers test

In [28]:
for i,name in enumerate(mutation_list):
    
    print(name)
    
    vis = 'binary'

    binary_header="""DATASET_BINARY

    SEPARATOR COMMA

    DATASET_LABEL,binary

    COLOR,#e41a1c

    FIELD_SHAPES,1

    FIELD_LABELS,f1

    DATA
    """

    binary_annotations = ''

    for i in tqdm(range(ts.num_nodes)):   

        row = new_tables.nodes[i]

        if 'name' in row.metadata.keys():

            uid = row.metadata['name']

            if uid in LABELS[LABELS[name]].index:
                line = 'n'+str(i) +',1\n'
                binary_annotations += line 

    binary_annotations = binary_header + binary_annotations
    
    title = 'ct-CM-annotations_' + name + '_circle.txt'
    print(title)

    with open(title,'w') as f:
        f.write(binary_annotations)

sigA_L386M


100%|███████████████████████████████████| 30454/30454 [00:03<00:00, 9180.32it/s]


ct-CM-annotations_sigA_L386M_circle.txt
sigA_G380A


100%|███████████████████████████████████| 30454/30454 [00:03<00:00, 8044.15it/s]


ct-CM-annotations_sigA_G380A_circle.txt
rpoC_I128V


...

ct-CM-annotations_rpoB_R791T_circle.txt
rpoB_E460D


100%|███████████████████████████████████| 30454/30454 [00:04<00:00, 7573.62it/s]


ct-CM-annotations_rpoB_E460D_circle.txt
sigA_E385Q


100%|███████████████████████████████████| 30454/30454 [00:03<00:00, 8211.95it/s]

ct-CM-annotations_sigA_E385Q_circle.txt


In [150]:
tree = ts.first()

newick_string = tree.as_newick()

'(n1:6.90799999999990177,n2:3.09200000000009823,(n4:0.46453999999994267,(n6:8.18533999999999651,(((n12:9.91015000000015789,n13:8.08985000000006949):0.17857000000003609,((((n20:1.73206000000004678,n21:4.26793999999995322):0.87799999999992906,(n22:0.00099999999997635,n23:2.00000000000000000):3.62199999999984357):8.94817000000011831,(n24:7.96229000000016640,(n26:5.42753000000016073,(n28:1.00000000000000000,n29:0.00099999999997635):3.07247000000006665):2.53771000000006097):1.48932999999988169):0.11167999999997846,(((n34:0.89059000000020205,(n36:1.04859000000010383,n37:1.95141000000012355):2.10941000000002532):12.25711999999998625,(n38:12.04489999999987049,(n40:6.05602000000021690,(n42:7.91893000000027314,(n44:0.86409999999978027,n45:5.13589999999999236):0.08107000000018161):0.19398000000001048):0.70509999999967476):2.83663000000024113):0.34405999999989945,(n46:4.74698000000012144,((n50:6.11341000000015811,n51:8.88659000000006927):0.02957999999989624,((n54:0.00099999999997635,(n56:6.1057700

Make a bar plot for the growth corresponding to each sample

In [85]:
vis = 'bar'

bar_header="""DATASET_SIMPLEBAR

SEPARATOR COMMA

DATASET_LABEL,bar

COLOR,#ff0000

WIDTH,100

DATA
"""

#ID1,value1
#ID2,value2
#9606,10000
#LEAF1|LEAF2,11000

bar_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']
        
        if numpy.any(PLATES[PLATES.UNIQUEID==LABELS.loc[uid].UNIQUEID].IM_POS_AVERAGE.values):
            
            growth = PLATES[PLATES.UNIQUEID==LABELS.loc[uid].UNIQUEID].IM_POS_AVERAGE.values[0]
            line = 'n'+ str(i) + ',' + str(growth) + '\n'
            bar_annotations += line 
        
        else:
            
            continue

#         if uid in LABELS[LABELS['HAS_T853A']].index:
#             line = 'n'+str(i) +',1\n'
#             bar_annotations += line 

bar_annotations = bar_header + bar_annotations


100%|████████████████████████████████████| 30454/30454 [01:35<00:00, 317.89it/s]


In [175]:
uid = new_tables.nodes[2].metadata['name']
GENOMES[GENOMES.TB_TYPE_1 == 'XDR']

#load UNIQUEIDs of resistant samples with CMs

res_CM_samples = []

for i, name in enumerate(hits_final.resistant_mutation.unique()):
    resistance = MUTATIONS[(MUTATIONS.GENE_MUTATION == name)].UNIQUEID
    hit_samples_local = MUTATIONS[(MUTATIONS.GENE_MUTATION.isin(hits_final.other_mutation.values.tolist())) 
                                  & (MUTATIONS.UNIQUEID.isin(resistance.tolist()))].UNIQUEID.unique()
    res_CM_samples.append(hit_samples_local)

res_CM_samples = numpy.concatenate(res_CM_samples)
res_CM_samples = pandas.Series(res_CM_samples)
res_CM_samples = res_CM_samples.tolist()

print(len(GENOMES[((GENOMES.TB_TYPE_1 == 'XDR')|(GENOMES.TB_TYPE_1 == 'MDR')) & (GENOMES.MYKROBE_LINEAGE_NAME_1 == 'Lineage 2') & (GENOMES.index.isin(res_CM_samples))])/len(GENOMES[((GENOMES.TB_TYPE_1 == 'XDR')|(GENOMES.TB_TYPE_1 == 'MDR')) & (GENOMES.MYKROBE_LINEAGE_NAME_1 == 'Lineage 2')]))
#GENOMES.index

0.4973363324257133


Make a colored strip for resistance type (RIF/ MDR/ XDR/ UNK/ SUS)

In [22]:
vis = 'colorstrip'

resistance_header="""DATASET_COLORSTRIP

SEPARATOR COMMA

DATASET_LABEL,label1

COLOR,#e41a1c

COLOR_BRANCHES,0

DATA
"""

colour_lookup = {1: '#F2CD5C', 2: '#F2921D', 3: '#A61F69', 4: '#400E32', 5: '#D3D3D3'}

resistance_annotations = ''

for i in tqdm(range(ts.num_nodes)): 

    row = new_tables.nodes[i]

    if 'name' in row.metadata.keys():

        uid = row.metadata['name']
        
        res_type = GENOMES.TB_TYPE_1[LABELS.loc[uid].UNIQUEID]

        if res_type == 'SUS':
            res_colour = colour_lookup[1]
            
        elif res_type == 'RIF':
            res_colour = colour_lookup[3]
            
        elif res_type == 'MDR':
            res_colour = colour_lookup[3]
            
        elif res_type == 'XDR':
            res_colour = colour_lookup[3]
            
        elif res_type == 'UNK':
            res_colour = colour_lookup[5]

        else:
            colour = '#bbbbbb'

        line = 'n'+ str(i) + ',' + res_colour + ',Resistance ' + res_type + '\n'
        resistance_annotations += line 
    

resistance_annotations = resistance_header + resistance_annotations


100%|███████████████████████████████████████████████████████████████████████████████| 30454/30454 [00:27<00:00, 1120.46it/s]


Write out the Tree in Newick format (probably don't need as we read it in earlier..) and the matching annotation files that you can drag and drop onto ITOL

In [24]:
# with open('ct-tree.newick','w') as f:
#     f.write(newick_string)

# with open('ct-prune-sparse.txt','w') as f:
#     f.write(prune_annotations)

# with open('ct-lineage-annotations-circle.txt','w') as f:
#     f.write(lineage_annotations)

# with open('ct-lineage-annotations-circle.txt','w') as f:
#     f.write(lineage_annotations)

# with open('ct-lineage-annotations-range.txt','w') as f:
#     f.write(lineage_annotations)

# with open('ct-annotations.txt','w') as f:
#     f.write(binary_annotations)
    
# with open('ct-resistant-annotations.txt','w') as f:
#     f.write(binary_annotations)
    
# with open('T853A_ct-CM-annotations_circle.txt','w') as f:
#     f.write(binary_annotations)

# with open('ct-mutation-annotations_V483G.txt','w') as f:
#     f.write(mutation_annotations)

# with open('ct-mutation-annotations-CMs-colored.txt','w') as f:
#     f.write(mutation_annotations)

# with open('ct-growth-bar.txt','w') as f:
#     f.write(bar_annotations)
    
with open('ct-resistance_type.txt','w') as f:
    f.write(resistance_annotations)

**Below are investigations of specific sub clades with either interesting aspects or accumulated CMs**

Investigate nodes with CMs arising before resistance

In [121]:
#n8190 - n8900

interesting_clade = []

for i in new_tables.nodes[8190:8900]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        interesting_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)
    
df = pandas.DataFrame(interesting_clade)
df

,0
0,rpoA_a-40o
1,rpoB_A1075A
2,rpoC_E1092D
3,rpoC_L1307L
4,rpoA_g-100x
...,...
5795,rpoC_E1092D
5796,rpoB_S450L
5797,rpoB_A1075A
5798,rpoC_I491T


In [122]:
df.value_counts()[0:50]

rpoB_A1075A    357
rpoC_E1092D    356
rpoB_S450L     225
rpoA_a-40o     157
rpoA_c-92x      83
rpoA_g-63x      83
rpoA_g-61x      83
rpoA_g-58x      83
rpoA_g-52x      83
rpoA_g-100x     83
rpoA_c-91x      83
rpoA_g-65x      83
rpoA_c-89x      83
rpoA_c-88x      83
rpoA_c-87x      83
rpoA_c-85x      83
rpoA_g-64x      83
rpoA_g-66x      83
rpoA_c-78x      83
rpoA_g-67x      83
rpoA_g-81x      83
rpoA_g-83x      83
rpoA_g-90x      83
rpoA_g-93x      83
rpoA_g-95x      83
rpoA_g-96x      83
rpoA_g-97x      83
rpoA_g-98x      83
rpoA_g-99x      83
rpoA_t-69x      83
rpoA_t-80x      83
rpoA_c-84x      83
rpoA_c-94x      83
rpoA_c-77x      83
rpoA_c-62x      83
rpoA_a-73x      83
rpoA_a-79x      83
rpoA_a-82x      83
rpoA_c-53x      83
rpoA_c-54x      83
rpoA_c-55x      83
rpoA_c-76x      83
rpoA_c-56x      83
rpoA_c-57x      83
rpoA_c-59x      83
rpoA_c-60x      83
rpoA_c-86x      83
rpoA_c-75x      83
rpoA_c-68x      83
rpoA_c-70x      83
dtype: int64

Interesting clade of lineage 4 clusters with lineage 5 and 6

In [30]:
#n15072, n15074, n15077, n15076
list = [15072, 15074, 15077, 15076]
interesting_clade2 = []

for i in list:
    node = new_tables.nodes[i]
    if 'name' in node.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == node.metadata['name']].index
        print(index[0])
        interesting_clade2.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)
    
df = pandas.DataFrame(interesting_clade2)

site.06.subj.06TB_0666.lab.06MIL1604.iso.1
site.06.subj.06TB_1029.lab.06MIL2034.iso.1
site.06.subj.06TB_1408.lab.06MIL2686.iso.1
site.06.subj.06TB_1406.lab.06MIL2684.iso.1


In [28]:
df.value_counts()

rpoA_a-40o     4
rpoA_g-67x     4
rpoA_c-92x     4
rpoA_c-94x     4
rpoA_g-100x    4
rpoA_g-48x     4
rpoA_g-50x     4
rpoA_g-52x     4
rpoA_g-58x     4
rpoA_g-61x     4
rpoA_g-63x     4
rpoA_g-64x     4
rpoA_g-65x     4
rpoA_g-66x     4
rpoA_g-81x     4
rpoA_a-42x     4
rpoA_g-83x     4
rpoA_g-90x     4
rpoA_g-93x     4
rpoA_g-95x     4
rpoA_g-96x     4
rpoA_g-97x     4
rpoA_g-98x     4
rpoA_g-99x     4
rpoA_t-49x     4
rpoA_t-69x     4
rpoA_t-80x     4
rpoB_A1075A    4
rpoA_c-91x     4
rpoA_c-89x     4
rpoA_c-88x     4
rpoA_c-87x     4
rpoA_a-73x     4
rpoA_a-79x     4
rpoA_a-82x     4
rpoA_c-41x     4
rpoA_c-43x     4
rpoA_c-53x     4
rpoA_c-54x     4
rpoA_c-55x     4
rpoA_c-56x     4
rpoA_c-57x     4
rpoA_c-59x     4
rpoA_c-60x     4
rpoA_c-62x     4
rpoA_c-68x     4
rpoA_c-70x     4
rpoA_c-71x     4
rpoA_c-72x     4
rpoA_c-74x     4
rpoA_c-75x     4
rpoA_c-76x     4
rpoA_c-77x     4
rpoA_c-78x     4
rpoA_c-84x     4
rpoA_c-85x     4
rpoA_c-86x     4
rpoC_V626I     1
dtype: int64

Interesting clade within lineage 2 (root): blue nodes have S450O instead of S450L!

In [171]:
#n1-n600
interesting_clade3 = []

for i in new_tables.nodes[1:600]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        interesting_clade3.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)
    
df = pandas.DataFrame(interesting_clade3)
df

,0
0,rpoB_S450O
1,rpoB_A1075A
2,rpoC_I491V
3,rpoB_S450L
4,rpoB_A1075A
...,...
4889,rpoA_a-40o
4890,rpoB_S450L
4891,rpoB_A1075A
4892,rpoC_G388G


In [46]:
df.value_counts()[:40]

rpoB_A1075A    299
rpoC_I491V     293
rpoB_S450L     286
rpoA_a-40o     118
rpoA_g-66x      72
rpoA_c-88x      72
rpoA_c-89x      72
rpoA_c-92x      72
rpoA_c-94x      72
rpoA_g-100x     72
rpoA_g-52x      72
rpoA_g-58x      72
rpoA_g-61x      72
rpoA_g-63x      72
rpoA_g-64x      72
rpoA_g-65x      72
rpoA_g-81x      72
rpoA_g-67x      72
rpoA_c-85x      72
rpoA_g-83x      72
rpoA_g-90x      72
rpoA_g-93x      72
rpoA_g-95x      72
rpoA_g-96x      72
rpoA_g-97x      72
rpoA_g-98x      72
rpoA_g-99x      72
rpoA_t-69x      72
rpoA_t-80x      72
rpoA_c-86x      72
rpoA_c-91x      72
rpoA_c-84x      72
rpoA_c-70x      72
rpoA_a-73x      72
rpoA_a-79x      72
rpoA_a-82x      72
rpoA_c-53x      72
rpoA_c-54x      72
rpoA_c-55x      72
rpoA_c-56x      72
dtype: int64

In [26]:
SAMPLES = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/SAMPLES.pkl.gz')
SAMPLES.COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()
SAMPLES.reset_index(inplace=True)

def create_uid(row):

    return 'site.' + row.SITEID + '.subj.' + row.SUBJID + '.lab.' + row.LABID + '.iso.1'

SAMPLES['UNIQUEID'] = SAMPLES.apply(create_uid, axis=1)

SAMPLES

,SITEID,SUBJID,LABID,COUNTRY_WHERE_SAMPLE_TAKEN,REGION,COLLECTION_DATE,ISOLATE_COLLECTED_PROSPECTIVELY,ANATOMICAL_ORIGIN,SMEAR_RESULT,WGS_SEQUENCING_PLATFORM,XPERT_MTB_RIF,HAIN_RIF,HAIN_INH,HAIN_FL,HAIN_AM,HAIN_ETH,SMOKER,INJECT_DRUG_USER,IS_HOMELESS,IS_IMPRISONED,HIV,DIABETES,WHO_OUTCOME,UNIQUEID
0,02,0958,22A197,CHN,ChongQing,2017-12-04 00:00:00+07:00,False,not known,not known,HiSeq,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.02.subj.0958.lab.22A197.iso.1
1,02,0823,2013241494,CHN,GuiZhou,2013-10-06 00:00:00+07:00,False,not known,not known,HiSeq,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.02.subj.0823.lab.2013241494.iso.1
2,02,0359,222018-14,CHN,ChongQing,2014-01-01 00:00:00+07:00,False,not known,not known,HiSeq,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.02.subj.0359.lab.222018-14.iso.1
3,02,0224,2013221088,CHN,ChongQing,2013-01-01 00:00:00+07:00,False,not known,not known,HiSeq,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.02.subj.0224.lab.2013221088.iso.1
4,02,1347,2013154310,CHN,Shandong,2013-01-01 00:00:00+07:00,False,not known,not known,HiSeq,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.02.subj.1347.lab.2013154310.iso.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16664,06,06TB_0565,06MIL1243,ITA,nan,NaT,False,not known,not known,Unknown,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.06.subj.06TB_0565.lab.06MIL1243.iso.1
16665,06,06TB_0566,06MIL1244,ITA,nan,NaT,False,not known,not known,Unknown,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.06.subj.06TB_0566.lab.06MIL1244.iso.1
16666,06,ICK-1132,06MIL1248,PAK,nan,NaT,False,not known,not known,Unknown,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.06.subj.ICK-1132.lab.06MIL1248.iso.1
16667,06,06TB_0474,06MIL1249,ITA,nan,NaT,False,not known,not known,Unknown,not tested,not tested,not tested,not tested,not tested,not tested,not known,not known,False,False,not known,not known,not known,site.06.subj.06TB_0474.lab.06MIL1249.iso.1


Look at SAMPLES.COUNTRY_WHERE_SAMPLE_TAKEN and SAMPLES.REGION

Investigate clades with accumulated CMs for oversampling of outbreaks

In [96]:
#V483A, n6419 - n6878
v483a_clade = []
growth_v483a = []

for i in new_tables.nodes[6419:6878]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_v483a.extend(index)
        v483a_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)

df = pandas.DataFrame(v483a_clade)
df.value_counts()[:20]
growth_v483a

# df = pandas.DataFrame(growth_v483a)
# df.to_csv('subclade_V483A.csv', index = False)

['site.03.subj.DR-285.lab.IML-01190.iso.1',
 'site.03.subj.DS-64.lab.IML-01232.iso.1',
 'site.03.subj.DS-150.lab.IML-01256.iso.1',
 'site.04.subj.01225.lab.720110.iso.1',
 'site.04.subj.01423.lab.26568.iso.1',
 'site.04.subj.03248.lab.823504.iso.1',
 'site.04.subj.04775.lab.907378.iso.1',
 'site.03.subj.DS-36.lab.IML-01239.iso.1',
 'site.03.subj.DR-44.lab.IML-01077.iso.1',
 'site.03.subj.DR-79.lab.IML-01092.iso.1',
 'site.03.subj.DS-10.lab.IML-01203.iso.1',
 'site.04.subj.00991.lab.717123.iso.1',
 'site.03.subj.DR-281.lab.IML-01188.iso.1',
 'site.04.subj.01901.lab.732532.iso.1',
 'site.04.subj.03470.lab.827987.iso.1',
 'site.04.subj.04118.lab.832668.iso.1',
 'site.04.subj.01900.lab.801874.iso.1',
 'site.04.subj.03027.lab.823876.iso.1',
 'site.03.subj.DR-320.lab.IML-01200.iso.1',
 'site.03.subj.DR-91.lab.IML-01095.iso.1',
 'site.03.subj.DR-121.lab.IML-01105.iso.1',
 'site.04.subj.01318.lab.723498.iso.1',
 'site.04.subj.04502.lab.902838.iso.1',
 'site.04.subj.03305.lab.825625.iso.1',
 's

In [28]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_v483a)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

COUNTRY_WHERE_SAMPLE_TAKEN
IND    181
NPL     48
SWE      1
AFG      0
NGA      0
Name: count, dtype: int64

In [29]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_v483a)].REGION.value_counts()[:30]

REGION
Kathmandu                     48
Mumbai                        32
Mumbai (Malad)                 8
Mumbra                         6
Mumbai (Mazagaon)              6
Mumbai (Central)               5
Mumbai (Jogeshwari)            5
Mumbai (Kurla)                 4
Mumbai (Malad west)            4
Mumbai (Byculla)               4
Mumbai (Wadala)                4
Mumbai (Grant Road)            4
Mumbai (Mira Road)             4
Thane                          4
Mumbai (Sion)                  4
Mumbai (Bhandup)               3
Mira Bhayandar (Bhayandar)     3
Mumbai (Kurla west)            3
Mumbai (Andheri west)          3
Mumbai (Nagpada)               3
Mumbai (Ghatkopar)             3
Mumbai (Mahim)                 3
Mumbai (Andheri)               3
Nala Sopara                    2
Mumbai (Dadar)                 2
Mumbai (Dahisar)               2
Mumbai (Chunabhatti)           2
Mumbai (Borivali)              2
Mumbai (Saki Naka)             2
Mumbai (Govandi)               2
Nam

In [103]:

for i in growth_v483a:
    sub_lin = GENOMES[GENOMES.index == i].MYKROBE_LINEAGE_NAME_2
    print(sub_lin)

UNIQUEID
site.03.subj.DR-285.lab.IML-01190.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DS-64.lab.IML-01232.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DS-150.lab.IML-01256.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01225.lab.720110.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01423.lab.26568.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03248.lab.823504.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04775.lab.907378.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DS-36.lab.IML-01239.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DR-44.lab.IML-01077.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DR-79.lab.IML-01092.iso.1    lineag

UNIQUEID
site.04.subj.01550.lab.729588.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03324.lab.827121.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03263.lab.825133.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04900.lab.903336.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00755.lab.708950.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04909.lab.909510.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04192.lab.831568.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01465.lab.725621.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00392.lab.715620.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05477.lab.913495.iso.1    lineage2.2
Name: MYKROBE_

UNIQUEID
site.04.subj.03571.lab.829922.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01598.lab.726612.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05621.lab.918800.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01062.lab.719968.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00168.lab.700996.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00780.lab.706828.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00024.lab.631993.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00299.lab.702745.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DR-8.lab.IML-01060.iso.1    lineage2.2
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.GB-83220123.lab.GB-83220123.iso.1    lineage2.2
N

In [131]:
snp_distance = numpy.load('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/GPI_SNP_DISTANCES_VALUES.npy')
snp_label = numpy.load('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/GPI_SNP_DISTANCES_LABELS.npy')

print(len(snp_distance), len(snp_label))
print(snp_distance.shape)
len(snp_distance[(snp_distance<30) & (snp_distance!=0)])
print(len(snp_distance[(snp_distance<200) & (snp_distance!=0)])/(len(snp_distance)*len(snp_distance)))

15228 15228
(15228, 15228)
0.032514552120093985


In [251]:
snp_distances_clade1 = []
for i in growth_v483a:
    for j in growth_v483a:
        dist = snp_distance[snp_label == i][0][snp_label == j][0]
        snp_distances_clade1.append(dist)
    
snp_distances_clade1
df = pandas.DataFrame(snp_distances_clade1)
list_df = df.value_counts()

In [252]:
list_df = list_df.reset_index()

list_df.rename({0:'SNP_dist'}, axis = 1, inplace = True)

#remove SNP_dist = 0 since that equals the self-distance
index = list_df[list_df['SNP_dist'] == 0].index
list_df = list_df.drop(axis = 0, index = index)

list_df = list_df.reset_index()
list_df.drop(axis = 1, columns='index', inplace = True)

list_df

,SNP_dist,count
0,22,3328
1,21,3318
2,23,3214
3,20,3074
4,19,2886
...,...,...
140,142,2
141,144,2
142,145,2
143,148,2


In [253]:
sum_df = 0
total_count = 0

for i in range(0,len(list_df)):
    sum_df += list_df['SNP_dist'][i] * list_df['count'][i]
    total_count += list_df['count'][i] 
    
average_SNP_dist = sum_df/total_count
print('The average SNP-distance is', average_SNP_dist, 'for V483A')

The average SNP-distance is 30.659347695088883 for I491V


In [254]:
list_df['SNP_dist'].max()

149

In [250]:
#V483G, n10889 - n10982
v483g_clade = []
growth_v483g = []

for i in new_tables.nodes[10889:10982]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_v483g.extend(index)
        v483g_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)

df = pandas.DataFrame(v483g_clade)
df.value_counts()[:20]

# df = pandas.DataFrame(growth_v483g)
# df.to_csv('subclade_V483G.csv', index = False)

rpoB_A1075A    49
rpoB_S450L     49
rpoC_V483G     46
rpoA_a-40o     18
rpoC_V483A      2
rpoA_c-53x      1
rpoA_g-90x      1
rpoA_g-52x      1
rpoA_g-58x      1
rpoA_g-61x      1
rpoA_g-63x      1
rpoA_g-64x      1
rpoA_g-65x      1
rpoA_g-66x      1
rpoA_g-67x      1
rpoA_g-81x      1
rpoA_g-83x      1
rpoA_g-93x      1
rpoA_c-54x      1
rpoA_g-95x      1
Name: count, dtype: int64

In [129]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_v483g)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

ZAF    43
SWE     1
NGA     0
VNM     0
UKR     0
Name: COUNTRY_WHERE_SAMPLE_TAKEN, dtype: int64

In [130]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_v483g)].REGION.value_counts()[:30]

Eastern Cape     25
Western Cape     11
KwaZulu-Natal     5
Stockholm         1
South Africa      1
Gauteng           1
Name: REGION, dtype: int64

In [166]:
#I491T, n7646 - n7860
i491t_clade = []
growth_i491t = []

for i in new_tables.nodes[7646:7860]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_i491t.extend(index)
        i491t_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)

df = pandas.DataFrame(i491t_clade)
df.value_counts()

# #I491 T (V) n1143 - n1358
i491t_clade = []
growth_i491t = []

for i in new_tables.nodes[1143:1358]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_i491t.extend(index)
        i491t_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)

df = pandas.DataFrame(i491t_clade)
df.value_counts()

# df = pandas.DataFrame(growth_i491t)
# df.to_csv('subclade_I491t.csv', index = False)

In [137]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_i491t)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

IND    102
NPL      7
AFG      0
NGA      0
VNM      0
Name: COUNTRY_WHERE_SAMPLE_TAKEN, dtype: int64

In [138]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_i491t)].REGION.value_counts()[:30]

Mumbai                             16
Thane                              11
Kathmandu                           7
Nala Sopara                         6
Bhiwandi                            5
Mumbai (Kandivali)                  5
Mumbai (Ghatkopar)                  4
Mumbai (Mahim)                      3
Mumbra                              3
Mumbai (Dharavi)                    3
Mumbai (Tardeo)                     2
Mumbai (Charni Road)                2
Mumbai (Malad east)                 2
Mumbai (Parel)                      2
Ulhasnagar                          2
Mumbai (Chembur)                    2
Mumbai (Saki Naka)                  2
Thane (west)                        2
Vasai-Virar                         1
Mumbai (Govandi)                    1
Mumbai (Vidyavihar)                 1
Mumbai (Worli)                      1
Mumbai (Kandivali east)             1
Mumbai (Guru Teg Bahadur Nagar)     1
Mumbai (Sewri)                      1
Mumbai (Agripada)                   1
Nashik      

In [46]:
#I491V, n1 - n616
i491v_clade = []
growth_i491v = []

for i in new_tables.nodes[1:616]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_i491v.extend(index)
        i491v_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)

df = pandas.DataFrame(i491v_clade)
df.value_counts()

# df = pandas.DataFrame(growth_i491v)
# df.to_csv('subclade_I491V.csv', index = False)

rpoB_A1075A    304
rpoC_I491V     295
rpoB_S450L     290
rpoA_a-40o     120
rpoA_g-100x     73
              ... 
rpoA_g-66o       1
rpoA_c-77o       1
rpoA_c-71o       1
rpoA_D253O       1
sigA_G314S       1
Name: count, Length: 78, dtype: int64

In [47]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_i491v)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

COUNTRY_WHERE_SAMPLE_TAKEN
IND    293
NPL     10
AFG      0
NGA      0
VNM      0
Name: count, dtype: int64

In [48]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_i491v)].REGION.value_counts()[:30]

REGION
Mumbai                    61
Mumbai (Mahim)            25
Thane                     12
Kathmandu                 10
Mumbai (Kurla)             9
Mumbai (Malad)             7
Dombivli                   6
Mumbai (Malad west)        6
Mumbai (Kandivali)         6
Mumbai (Andheri)           6
Mumbra                     5
Mumbai (Wadala)            5
Mumbai (Central)           5
Mumbai (Parel)             5
Mumbai (Antop Hill)        4
Mumbai (Borivali)          4
Mumbai (Mazagaon)          4
Mumbai (Sewri)             4
Virar                      4
Mumbai (Chembur)           4
Mumbai (Bandra)            4
Mumbai (Sion)              4
Mumbai (Goregaon east)     3
Mumbai (Goregaon)          3
Mumbai (Bhandup)           3
Vasai-Virar                3
Mumbai (Prabhadevi)        3
Mumbai (Santacruz)         3
Mumbai (Girgaon)           3
Mumbai (Ghatkopar)         3
Name: count, dtype: int64

In [246]:
snp_distances_clade2 = []
for i in growth_i491v:
    for j in growth_i491v:
        dist = snp_distance[snp_label == i][0][snp_label == j][0]
        snp_distances_clade2.append(dist)
    
snp_distances_clade2
df = pandas.DataFrame(snp_distances_clade2)
list_df = df.value_counts()

In [104]:
for i in growth_i491v:
    sub_lin = GENOMES[GENOMES.index == i].MYKROBE_LINEAGE_NAME_2
    print(sub_lin)

UNIQUEID
site.04.subj.03818.lab.830476.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00310.lab.701951.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01627.lab.27972.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01042.lab.717652.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.02112.lab.803805.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04137.lab.832790.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01432.lab.724551.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.02235.lab.805177.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00300.lab.702487.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03648.lab.JJH9682.iso.1    lineage

UNIQUEID
site.04.subj.03824.lab.830216.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05660.lab.919709.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00482.lab.704543.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03893.lab.832777.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00080.lab.633839.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05005.lab.908704.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03363.lab.827295.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01877.lab.801709.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03870.lab.826149.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04918.lab.909362.iso.1    lineage

UNIQUEID
site.04.subj.03224.lab.822877.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.02062.lab.21190.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05614.lab.918016.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00998.lab.715906.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03211.lab.825251.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04174.lab.833074.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01682.lab.732653.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03995.lab.832048.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01002.lab.717763.iso.1    lineage2.2.7
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03741.lab.826347.iso.1    lineage2

In [105]:
#P1040R, n9922 - n10424
p1040r_clade = []
growth_p1040r = []

for i in new_tables.nodes[9922:10424]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_p1040r.extend(index)
        p1040r_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)
        
df = pandas.DataFrame(p1040r_clade)
df.value_counts()

df = pandas.DataFrame(growth_p1040r)
df.to_csv('subclade_P1040R.csv', index = False)

In [106]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_p1040r)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

COUNTRY_WHERE_SAMPLE_TAKEN
IND    242
NPL      8
AFG      0
NGA      0
VNM      0
Name: count, dtype: int64

In [107]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_p1040r)].REGION.value_counts()[:30]

REGION
Mumbai                 56
Thane                  14
Kalyan                 12
Mumbai (Mahim)         10
Mumbai (Malad)          8
Kathmandu               8
Mumbai (Santacruz)      7
Mumbai (Wadala)         6
Mumbai (Dadar)          5
Mumbai (Dharavi)        5
Nala Sopara             5
Bandra (east)           4
Mumbai (Mazagaon)       4
Mumbai (Kurla)          4
Mumbra                  4
Mumbai (Goregaon)       4
Mumbai (Bandra)         4
Mumbai (Andheri)        3
Mumbai (Malad west)     3
Mumbai (Parel)          3
Ulhasnagar              3
Mumbai (Mulund)         3
Mumbai (Jogeshwari)     3
Mumbai (Ghatkopar)      3
Mumbai (Sion)           3
Bhiwandi                3
Mumbai (Prabhadevi)     2
Mumbai (Govandi)        2
Mumbai (Vile Parle)     2
Dombivli                2
Name: count, dtype: int64

In [109]:
for i in growth_p1040r:
    sub_lin = GENOMES[GENOMES.index == i].MYKROBE_LINEAGE_NAME_2
    print(sub_lin)

UNIQUEID
site.04.subj.04787.lab.904613.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00506.lab.709045.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05418.lab.913041.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00942.lab.715954.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03168.lab.821506.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04407.lab.903102.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03864.lab.830782.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.02158.lab.732526.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03528.lab.829149.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03192.lab.816831.iso.1    lineage

UNIQUEID
site.04.subj.03478.lab.827844.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04852.lab.907752.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00620.lab.707489.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00757.lab.714670.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.04115.lab.832555.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DS-24.lab.IML-01207.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01711.lab.729885.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.03.subj.DR-161.lab.IML-01125.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00966.lab.716955.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01788.lab.732893.iso.1    

UNIQUEID
site.04.subj.00781.lab.708146.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05576.lab.919703.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00997.lab.718598.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00769.lab.710846.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.00524.lab.706416.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.05639.lab.919649.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01589.lab.730725.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.03340.lab.826105.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01480.lab.729217.iso.1    lineage2.2.3
Name: MYKROBE_LINEAGE_NAME_2, dtype: object
UNIQUEID
site.04.subj.01508.lab.725271.iso.1    lineage

In [159]:
#G332S n5215 - n5392
g332s_clade = []
growth_g332s = []

for i in new_tables.nodes[5215:5392]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_g332s.extend(index)
        g332s_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)
        
df = pandas.DataFrame(g332s_clade)
df.value_counts()

#df = pandas.DataFrame(growth_g332s)
#df.to_csv('subclade_G332S.csv', index = False)

In [147]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_g332s)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

IND    88
NPL     1
AFG     0
NGA     0
VNM     0
Name: COUNTRY_WHERE_SAMPLE_TAKEN, dtype: int64

In [149]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_g332s)].REGION.value_counts()[:30]

Mumbai                        23
Mumbai (Mahim)                13
Virar                          4
Mumbai (Kandivali)             4
Mumbai (Ghatkopar)             4
Mumbai (Grant Road)            2
Mumbai (Chembur)               2
Thane                          2
Mumbai (Wadala)                2
Mumbai (Sion)                  2
Nala Sopara                    2
Mumbai (Marine Line)           2
Mira Bhayandar (Bhayandar)     1
Mumbai (Lower Parel)           1
Mumbai (Chinchpokli)           1
Mumbai (Central)               1
Dapoli                         1
Pant Nagar Ghat                1
Panvel                         1
Mumbai (Santacruz)             1
Mumbai (Mankhurd)              1
Mumbai (Mulund)                1
Goa                            1
Mumbai (Byculla)               1
Uran                           1
Navi Mumbai (Kharghar)         1
Mumbai (Kurla east)            1
Mumbai (Worli)                 1
Mumbai (Sewri)                 1
Mumbai (Charni Road)           1
Name: REGI

In [162]:
#G433S n940 - n1076
g433s_clade = []
growth_g433s = []

for i in new_tables.nodes[940:1076]:
    if 'name' in i.metadata.keys():
        index = GENOMES[GENOMES['ORIGINAL_UID'] == i.metadata['name']].index
        growth_g433s.extend(index)
        g433s_clade.extend(MUTATIONS[MUTATIONS.UNIQUEID == index[0]].GENE_MUTATION.values)
        
df = pandas.DataFrame(g433s_clade)
df.value_counts()

#df = pandas.DataFrame(growth_g433s)
#df.to_csv('subclade_G433S.csv', index = False)

In [151]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_g433s)].COUNTRY_WHERE_SAMPLE_TAKEN.value_counts()[:5]

IND    61
NPL     8
AFG     0
NGA     0
VNM     0
Name: COUNTRY_WHERE_SAMPLE_TAKEN, dtype: int64

In [152]:
SAMPLES[SAMPLES.UNIQUEID.isin(growth_g433s)].REGION.value_counts()[:30]

Mumbai                     16
Kathmandu                   8
Kalyan                      5
Mumbai (Mahim)              5
Mumbai (Bhandup)            4
Mumbai (Andheri)            3
Mumbai (Girgaon)            2
Mumbai (Kurla)              2
Mumbai (Wadala)             2
Ulhasnagar                  2
Mumbai (Charni Road)        1
Mumbai (Sewri)              1
Mumbai (Govandi)            1
Mumbai (Parel)              1
Navi Mumbai (Vashi)         1
Mumbra                      1
Dombivli                    1
Mumbai (Vikhroli)           1
Mumbai (Khar)               1
Ambivli                     1
Mira Bhayandar (Dongri)     1
Mumbai (Chembur)            1
Kalwa (west)                1
Thane (east)                1
Mumbai (Mazagaon)           1
Ratnagiri                   1
Mumbai (Malad)              1
Mumbai (Sion)               1
Badlapur                    1
Mumbai (Dharavi)            1
Name: REGION, dtype: int64